<font color='blue'><h1>Projeto de Insights</h1></font>

<font color='red'><h3>Questão de Negócio</h3></font>

1. Quais são os imóveis que a House Rocket deveria comprar e por qual preço?
2. Uma vez o imóvel foi comprado, qual o melhor momento para vendê-lo e por qual preço?

### Hipoteses

- H1: Imóveis que possuem vista para a água, são 30% mais caros, na média; 
- H2: Imóveis com data de construção menor que 1955, são 50% mais baratos, na média; 
- H3: Imóveis sem porão possuem área total (sqrt_lot), são 40% maiores do que os imóveis com porão; 
- H4: Imóveis com 3 banheiros são 50% mais caros do que os imóveis com 2 banheiros. 

<font color='red'><h3>Validando as Hipóteses</h3></font>

In [2]:
import pandas as pd 
import numpy as np

path = 'kc_house_data.csv'
df = pd.read_csv(path)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

### H1 - Verdadeira? R: São 3.13x mais caros</font>

In [36]:
total = df['id'].size

total_with = df.loc[df['waterfront'] == 1, 'id'].size

avg_with = round(df.loc[df['waterfront'] == 1, 'price'].mean(), 2)

total_without = df.loc[df['waterfront'] != 1, 'id'].size

avg_without = df.loc[df['waterfront'] != 1, 'price'].mean()

if avg_with > avg_without: 
    print(f'Em média os imóveis com vista para a água são {round((avg_with/avg_without), 2)}x mais caros do que os sem vista')

else: 
    print('Imóveis com vista para a água são mais baratos')

Em média os imóveis com vista para a água são 3.13x mais caros do que os sem vista


### H2 - Verdadeira? R: Os imóveis mais velhos são 0.78% mais baratos

In [56]:
total = df['yr_built'].size

# Filtro dos imóveis mais novos que 1955
new_properties = df.loc[(df['yr_built'].astype('str')) >= '1955']
avg_price_new = round(new_properties['price'].mean(), 2)

# Filtro dos imóveis mais velhos que 1955
old_properties = df.loc[(df['yr_built'].astype('str')) < '1955']
avg_price_old = round(old_properties['price'].mean(), 2)

print(f'Média dos preços de imóveis: \n  - Mais antigos que 1995: {avg_price_old}\n  - Imóveis de 1995 até mais atual: {avg_price_new}')

avg_price_percent = round((1 - (avg_price_old / avg_price_new)) * 100, 2)
print(f'Os imóveis antigos são mais baratos do que os novos em: {avg_price_percent}%')

Média dos preços de imóveis: 
  - Mais antigos que 1995: 537050.91
  - Imóveis de 1995 até mais atual: 541299.97
Os imóveis antigos são mais baratos do que os novos em: 0.78%


### H3 - Verdadeira? R: Os imóveis sem porão são 22.56% maiores do que os com porão!

In [65]:
with_basement = df.loc[df['sqft_basement'] != 0]
sqft_lot_with = round(with_basement['sqft_lot'].mean(), 2)

without_basement = df.loc[df['sqft_basement'] == 0]
sqft_lot_without = round(without_basement['sqft_lot'].mean(), 2)

percent = round(((sqft_lot_without / sqft_lot_with) - 1) * 100, 2)
print(f'Os imóveis sem porão são {percent}% maiores do que os com porão!')

Os imóveis sem porão são 22.56% maiores do que os com porão!


### H4 - Verdadeira? R: Os imóveis com 3 banheiros são 54.71% mais caros do que os que possuem 2 imóveis.

In [130]:
mean_3bath = df.loc[df['bathrooms'] == 3, 'price'].mean()
mean_2bath = df.loc[df['bathrooms'] == 2, 'price'].mean()

percent = round(((mean_3bath/mean_2bath) - 1) * 100, 2)
print(f'Imóveis com 3 banheiros são {percent}% mais caros do que os que possuem 2 imóveis')

Imóveis com 3 banheiros são 54.71% mais caros do que os que possuem 2 imóveis
